
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session.                                                                                                 |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0).                               |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |  Changes the session type to Glue ETL.                                                                                                                    |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X.                                                                           |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer.                      |

In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::162504240738:role/AWSGlueServiceNotebookRoleDefault
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 50c92d9f-12b1-4354-a240-075197383039
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
Waiting for session 50c92d9f-12b1-4354-a240-075197383039 to get into ready status...
Session 50c92d9f-12b1-4354-a240-075197383039 has been created.



In [2]:
university_ranking_source = glueContext.create_dynamic_frame.from_catalog(
    database="learn_by_doing",
    table_name="university_ranking_csv",
    transformation_ctx="UniversityRanking_Source",
)

In [4]:
university_ranking_source.show(5)

{"university": "Massachusetts Institute of Technology (MIT) ", "year": "2017", "rank_display": "1", "score": "100.0", "link": "https://www.topuniversities.com/universities/massachusetts-institute-technology-mit", "country": "United States", "city": "Cambridge", "region": "North America", "logo": "https://www.topuniversities.com/sites/default/files/massachusetts-institute-of-technology-mit_410_small.jpg", "type": "Private", "research_output": "Very High", "student_faculty_ratio": "4.0", "international_students": "3,730", "size": "M", "faculty_count": "3,065"}
{"university": "Stanford University", "year": "2017", "rank_display": "2", "score": "98.7", "link": "https://www.topuniversities.com/universities/stanford-university", "country": "United States", "city": "Stanford", "region": "North America", "logo": "https://www.topuniversities.com/sites/default/files/stanford-university_573_small.jpg", "type": "Private", "research_output": "Very High", "student_faculty_ratio": "3.0", "internation

In [5]:
rankings_df = university_ranking_source.toDF()

In [6]:
rankings_df.show(5)

+--------------------+----+------------+-----+--------------------+--------------+---------+-------------+--------------------+-------+---------------+---------------------+----------------------+----+-------------+
|          university|year|rank_display|score|                link|       country|     city|       region|                logo|   type|research_output|student_faculty_ratio|international_students|size|faculty_count|
+--------------------+----+------------+-----+--------------------+--------------+---------+-------------+--------------------+-------+---------------+---------------------+----------------------+----+-------------+
|Massachusetts Ins...|2017|           1|100.0|https://www.topun...| United States|Cambridge|North America|https://www.topun...|Private|      Very High|                  4.0|                 3,730|   M|        3,065|
| Stanford University|2017|           2| 98.7|https://www.topun...| United States| Stanford|North America|https://www.topun...|Private| 

In [7]:
rankings_df.createOrReplaceTempView("university_ranking")

In [9]:
spark.sql("""SELECT university,year,rank_display,region,country FROM university_ranking""").show(5)

+--------------------+----+------------+-------------+--------------+
|          university|year|rank_display|       region|       country|
+--------------------+----+------------+-------------+--------------+
|Massachusetts Ins...|2017|           1|North America| United States|
| Stanford University|2017|           2|North America| United States|
|  Harvard University|2017|           3|North America| United States|
|University of Cam...|2017|           4|       Europe|United Kingdom|
|California Instit...|2017|           5|North America| United States|
+--------------------+----+------------+-------------+--------------+
only showing top 5 rows


In [13]:
spark.sql("""SELECT university,year,rank_display, int(rank_display) as n_rank,region,country 
FROM university_ranking""").show(5)

+--------------------+----+------------+------+-------------+--------------+
|          university|year|rank_display|n_rank|       region|       country|
+--------------------+----+------------+------+-------------+--------------+
|Massachusetts Ins...|2017|           1|     1|North America| United States|
| Stanford University|2017|           2|     2|North America| United States|
|  Harvard University|2017|           3|     3|North America| United States|
|University of Cam...|2017|           4|     4|       Europe|United Kingdom|
|California Instit...|2017|           5|     5|North America| United States|
+--------------------+----+------------+------+-------------+--------------+
only showing top 5 rows


## Split the rank_display data and get the first value

In [15]:
spark.sql("""SELECT university,year,rank_display, int(split(rank_display,'-')[0]) as n_rank,region,country 
FROM university_ranking
WHERE rank_display like '%-%'""").show(5)


+--------------------+----+------------+------+-------------+--------------------+
|          university|year|rank_display|n_rank|       region|             country|
+--------------------+----+------------+------+-------------+--------------------+
|Bandung Institute...|2017|     401-410|   401|         Asia|           Indonesia|
| Brandeis University|2017|     401-410|   401|North America|       United States|
|Illinois Institut...|2017|     401-410|   401|North America|       United States|
|Johannes Gutenber...|2017|     401-410|   401|       Europe|             Germany|
|Khalifa Universit...|2017|     401-410|   401|         Asia|United Arab Emirates|
+--------------------+----+------------+------+-------------+--------------------+
only showing top 5 rows


In [16]:
# Works for all rows
spark.sql("""SELECT university,year,rank_display, int(split(rank_display,'-')[0]) as n_rank,region,country 
FROM university_ranking""").show(5)

+--------------------+----+------------+------+-------------+--------------+
|          university|year|rank_display|n_rank|       region|       country|
+--------------------+----+------------+------+-------------+--------------+
|Massachusetts Ins...|2017|           1|     1|North America| United States|
| Stanford University|2017|           2|     2|North America| United States|
|  Harvard University|2017|           3|     3|North America| United States|
|University of Cam...|2017|           4|     4|       Europe|United Kingdom|
|California Instit...|2017|           5|     5|North America| United States|
+--------------------+----+------------+------+-------------+--------------+
only showing top 5 rows


In [19]:
spark.sql("""SELECT 
    university,
    coalesce(int(year),9999) as year,
    rank_display,
    coalesce(int(split(rank_display,'-')[0]),9999) as n_rank,
    coalesce(float(score),-1) as score,
    country, city, region, type,
    research_output,
    coalesce(float(student_faculty_ratio),-1) as student_faculty_ratio,
    coalesce(int(regexp_replace(international_students,'[.,]','')),-1) as international_students,
    size,
    coalesce(int(regexp_replace(faculty_count,'[.,]','')),-1) as faculty_count
    FROM university_ranking""").show(5)

+--------------------+----+------------+------+-----+--------------+---------+-------------+-------+---------------+---------------------+----------------------+----+-------------+
|          university|year|rank_display|n_rank|score|       country|     city|       region|   type|research_output|student_faculty_ratio|international_students|size|faculty_count|
+--------------------+----+------------+------+-----+--------------+---------+-------------+-------+---------------+---------------------+----------------------+----+-------------+
|Massachusetts Ins...|2017|           1|     1|100.0| United States|Cambridge|North America|Private|      Very High|                  4.0|                  3730|   M|         3065|
| Stanford University|2017|           2|     2| 98.7| United States| Stanford|North America|Private|      Very High|                  3.0|                  3879|   L|         4725|
|  Harvard University|2017|           3|     3| 98.3| United States|Cambridge|North America|Pri

In [20]:
# Let's Store the results data frame

clean_rankings_df = spark.sql("""SELECT 
    university,
    coalesce(int(year),9999) as year,
    rank_display,
    coalesce(int(split(rank_display,'-')[0]),9999) as n_rank,
    coalesce(float(score),-1) as score,
    country, city, region, type,
    research_output,
    coalesce(float(student_faculty_ratio),-1) as student_faculty_ratio,
    coalesce(int(regexp_replace(international_students,'[.,]','')),-1) as international_students,
    size,
    coalesce(int(regexp_replace(faculty_count,'[.,]','')),-1) as faculty_count
    FROM university_ranking""")

In [23]:
# Convert back to DynamicFrame
clean_dynamic_frame = DynamicFrame.fromDF(clean_rankings_df, glueContext, "university_ranking_clean")

In [26]:
# Store the clean data back to S3
university_ranking_destn = glueContext.write_dynamic_frame.from_options(
    frame=clean_dynamic_frame,
    connection_type="s3",
    format="csv",
    connection_options={
        "path": "s3://aws-glue-chandra-us-east-2/university_ranking/csv_clean",
        "partitionKeys": [],
    },
    transformation_ctx="university_ranking_dest",
)

In [27]:
job.commit()